In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
    "env_name":"user",
    "pipeline_name":"pipeline0",
    "zone_name":"zone0",
    "X": "something"
}
step_params={
    "X": "something_else"
}
substep_params={
    "param1":"None1",
    "param2":"None2"}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, get_user, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
   
    outputs = 
    [
        { ENTITY_NAME: "entityA" },
        { ENTITY_NAME: "entityB" },
        { ENTITY_NAME: "entityC" }
    ],
    custom_inputs =
    [
        { ENTITY_NAME: "custom_data", ENTITY_PATH: f"/data/home/{get_user()}/custom_data" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
#5 read inputs 
custom_inputs = substep.custom_inputs()

custom_data = spark.read.parquet(custom_inputs.custom_data).to_pandas_on_spark()

In [ ]:
#6 make something to create artifacts

dfCustom = custom_data.to_spark()

In [ ]:
#7 write outputs
outputs = substep.outputs()

dfCustom.write.parquet(outputs.entityA, mode='overwrite')
dfCustom.write.parquet(outputs.entityB, mode='overwrite')
dfCustom.write.parquet(outputs.entityC, mode='overwrite')

In [ ]:
#8 stop spark
SinaraSpark.stop_session()